# 02 - Intro Semantic Kernel

## Multi Modal Completion 


![Image](../assets/ImageCompletion.jpg)

## Environment

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.38.0"
#r "nuget: DotNetEnv, 3.1.1"

using System.IO;
using DotNetEnv;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;

string configurationFile = "../config/config.env";
string assetsFolder = "../assets";

Env.Load(configurationFile);

string apiKey = Env.GetString("SK_OPENAI_APIKEY");
string endpoint = Env.GetString("SK_OPENAI_ENDPOINT");
string chatCompletionDeployment = Env.GetString("SK_OPENAI_CHATCOMPLETION_DEPLOYMENT_DEFAULT");

Console.WriteLine($"Configuration loaded...");

## Create Kernel


In [ ]:
Kernel kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        apiKey: apiKey, 
        endpoint: endpoint, 
        deploymentName: chatCompletionDeployment
    )
    .Build();

IChatCompletionService chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

Console.WriteLine($"Kernel created...");

## Multi Modal Completion

In [ ]:
string systemMessage = "You describe provided images!";
string prompt = "Describe the image.";
string imageDataUri = await File.ReadAllTextAsync(Path.Join(assetsFolder, "ImageCompletion.txt"));

ChatHistory chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemMessage);

ChatMessageContentItemCollection chatMessageContentItemCollection = new ChatMessageContentItemCollection();
chatMessageContentItemCollection.Add(new ImageContent(){DataUri = imageDataUri});
chatMessageContentItemCollection.Add(new TextContent(){Text = prompt});

chatHistory.AddUserMessage(chatMessageContentItemCollection);

ChatMessageContent chatMessageContent = await chatCompletionService.GetChatMessageContentAsync(
    chatHistory
);

Console.WriteLine($"Model response: \n\t {chatMessageContent.Content} ");
